# Creating dynamic flows

In this tutorial, you will:

- Learn how to create dynamic workflows.
- Understand the `detour`, `addition`, and `replace` options in the `Response` object.

The ability create dynamic workflows (i.e. jobs or workflows that launch other jobs or workflows) is
a particularly powerful usage pattern in Jobflow.


## The `Response(replace)` option


The main mechanism for creating dynamic jobs in Jobflow is through the `Response` object. We will demonstrate this below for a toy example where we:

1. Generate a list of numbers whose length is only determined at runtime.
2. Perform a toy operation on each number in the list.

While this is a trivial example, a similar usage is common in computational materials science (e.g. you might perform a calculation on a bulk structure, carve all possible surface slabs, and then perform a calculation on each slab). What makes this dynamic is that the number of jobs is only determined at runtime.


In [32]:
import warnings

warnings.filterwarnings("ignore", "Using `tqdm.autonotebook.tqdm`")

In [33]:
from random import randint
from jobflow import job, Flow, Response
from jobflow.managers.local import run_locally

@job
def make_list(a):
    return [a] * randint(2, 5)

@job
def add(a, b):
    return a + b

@job
def add_distributed(list_a):
    jobs = []
    for val in list_a:
        jobs.append(add(val, 1))
    
    flow = Flow(jobs)
    return Response(replace=flow)

job1 = make_list(2)
job2 = add_distributed(job1.output)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-07 23:00:08,664 INFO Started executing jobs locally
2023-06-07 23:00:08,667 INFO Starting job - make_list (eb3de3d3-52e8-4628-b508-9d91176d7cf9)
2023-06-07 23:00:08,671 INFO Finished job - make_list (eb3de3d3-52e8-4628-b508-9d91176d7cf9)
2023-06-07 23:00:08,675 INFO Starting job - add_distributed (3b1b1784-d027-4fb5-a29c-47004876981a)
2023-06-07 23:00:08,683 INFO Finished job - add_distributed (3b1b1784-d027-4fb5-a29c-47004876981a)
2023-06-07 23:00:08,688 INFO Starting job - add (60b5a452-dbae-4f8b-a6f7-f6706151f7f2)
2023-06-07 23:00:08,693 INFO Finished job - add (60b5a452-dbae-4f8b-a6f7-f6706151f7f2)
2023-06-07 23:00:08,695 INFO Starting job - add (8cde4aba-9162-4ad6-96e9-a412544859b8)
2023-06-07 23:00:08,700 INFO Finished job - add (8cde4aba-9162-4ad6-96e9-a412544859b8)
2023-06-07 23:00:08,704 INFO Starting job - add (4b823573-7852-4bd7-b8ee-dc145401021f)
2023-06-07 23:00:08,707 INFO Finished job - add (4b823573-7852-4bd7-b8ee-dc145401021f)
2023-06-07 23:00:08,710 INFO Start

c:\Users\asros\miniconda3\envs\cms\lib\site-packages\jobflow\utils\graph.py:53: UserWarning: Some jobs are not connected, their ordering may be random
  warnings.warn("Some jobs are not connected, their ordering may be random")


In [34]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

eb3de3d3-52e8-4628-b508-9d91176d7cf9 -> {1: Response(output=[2, 2, 2, 2, 2], detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
3b1b1784-d027-4fb5-a29c-47004876981a -> {1: Response(output=None, detour=None, addition=None, replace=<jobflow.core.flow.Flow object at 0x000001A3C9C7A5B0>, stored_data=None, stop_children=False, stop_jobflow=False)}
60b5a452-dbae-4f8b-a6f7-f6706151f7f2 -> {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
8cde4aba-9162-4ad6-96e9-a412544859b8 -> {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
4b823573-7852-4bd7-b8ee-dc145401021f -> {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
79fdde1c-1884-49a7-8ff2-d9ce449392b0 -> {1: Response(output=3, detour=None, addition=None, replace=None, stor

As sene above, there are several jobs that were run --- certainly more than the two we started with. The first job generates a list of 2's with a random length. The second job in the flow is what launches a job on each entry in the list. It is replaced by one job for each entry, hence it has no direct output. Then each newly generated job is run.


## The `Response(addition)` option


Beyond replacing a job with downstream jobs, there is also the option to add jobs to the current flow on-the-fly.

Here we will create a simple flow that:

1. Adds a value to a given number.
2. If the output is less than 10, do the addition again. Otherwise, stop.


In [35]:
@job
def add(a, b):
    return a + b

@job
def add_with_logic(a, b):
    if a < 10:
        return Response(addition=add(a, b))
    
job1 = add(1, 2)
job2 = add_with_logic(job1.output, 2)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-07 23:00:08,875 INFO Started executing jobs locally
2023-06-07 23:00:08,878 INFO Starting job - add (33dea617-3a80-4c71-b8c7-76d44d758e97)
2023-06-07 23:00:08,881 INFO Finished job - add (33dea617-3a80-4c71-b8c7-76d44d758e97)
2023-06-07 23:00:08,883 INFO Starting job - add_with_logic (97fd505e-f6a1-4a37-b5e1-33858ae77020)
2023-06-07 23:00:08,887 INFO Finished job - add_with_logic (97fd505e-f6a1-4a37-b5e1-33858ae77020)
2023-06-07 23:00:08,888 INFO Starting job - add (b4fe3128-09a9-4827-b9ee-5b8ef73f123f)
2023-06-07 23:00:08,889 INFO Finished job - add (b4fe3128-09a9-4827-b9ee-5b8ef73f123f)
2023-06-07 23:00:08,891 INFO Finished executing jobs locally


In [36]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

33dea617-3a80-4c71-b8c7-76d44d758e97 -> {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
97fd505e-f6a1-4a37-b5e1-33858ae77020 -> {1: Response(output=None, detour=None, addition=<jobflow.core.flow.Flow object at 0x000001A3C9FFCC10>, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
b4fe3128-09a9-4827-b9ee-5b8ef73f123f -> {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


As you can see above, the addition job is correctly run twice. Now let's confirm that the addition job is only run once if the output of the first job is greater than 10.


In [37]:
@job
def add(a, b):
    return a + b

@job
def add_with_logic(a, b):
    if a < 10:
        return Response(addition=add(a, b))
    
job1 = add(1, 20)
job2 = add_with_logic(job1.output, 20)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-07 23:00:09,079 INFO Started executing jobs locally
2023-06-07 23:00:09,081 INFO Starting job - add (0bfd030e-6caa-4b47-a6eb-9b0873044a61)
2023-06-07 23:00:09,083 INFO Finished job - add (0bfd030e-6caa-4b47-a6eb-9b0873044a61)
2023-06-07 23:00:09,085 INFO Starting job - add_with_logic (6c676626-595c-4cbb-82d2-71827e0bdfea)
2023-06-07 23:00:09,088 INFO Finished job - add_with_logic (6c676626-595c-4cbb-82d2-71827e0bdfea)
2023-06-07 23:00:09,091 INFO Finished executing jobs locally


In [38]:
for uuid, response in responses.items():
    print(f"{uuid} -> {response}")

0bfd030e-6caa-4b47-a6eb-9b0873044a61 -> {1: Response(output=21, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
6c676626-595c-4cbb-82d2-71827e0bdfea -> {1: Response(output=None, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


Now, we see that the `Response(addition)` does not launch a new job.


## The `Response(detour)` option


The `Response(detour)` option behaves similarly to `Response(addition)`. The difference is that `Response(addition)` will add a job (or flow) to the current flow, while `Response(detour)` will no longer run the current flow and will switch to a parallel job or flow.


In [39]:
@job
def add(a, b):
    return a + b

@job
def add_with_logic(a, b):
    if a < 10:
        return Response(detour=add(a, b))
    
job1 = add(1, 2)
job2 = add_with_logic(job1.output, 2)
flow = Flow([job1, job2])

responses = run_locally(flow)

2023-06-07 23:00:09,220 INFO Started executing jobs locally
2023-06-07 23:00:09,221 INFO Starting job - add (d7ac181e-8f9c-4383-a3a9-6559f9b37315)
2023-06-07 23:00:09,223 INFO Finished job - add (d7ac181e-8f9c-4383-a3a9-6559f9b37315)
2023-06-07 23:00:09,223 INFO Starting job - add_with_logic (f560ae22-82c2-425c-b67f-36b27c83373c)
2023-06-07 23:00:09,226 INFO Finished job - add_with_logic (f560ae22-82c2-425c-b67f-36b27c83373c)
2023-06-07 23:00:09,228 INFO Starting job - add (19823318-ec29-4ed4-9ef9-896bab616801)
2023-06-07 23:00:09,230 INFO Finished job - add (19823318-ec29-4ed4-9ef9-896bab616801)
2023-06-07 23:00:09,231 INFO Finished executing jobs locally


d7ac181e-8f9c-4383-a3a9-6559f9b37315 -> {1: Response(output=3, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
f560ae22-82c2-425c-b67f-36b27c83373c -> {1: Response(output=None, detour=<jobflow.core.flow.Flow object at 0x000001A3C9FEE5B0>, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}
19823318-ec29-4ed4-9ef9-896bab616801 -> {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}


For this toy example, both `Response(addition)` and `Response(detour)` behave identically.
